## Imports

In [ ]:
import datetime
import gc

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
cuda_use_gpus(1)

In [ ]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Config

In [ ]:
feature_list_id = 'oofp_ubernet'

In [ ]:
NUM_FOLDS = 5
BATCH_SIZE = 512
NUM_EPOCHS = 200

In [ ]:
RANDOM_SEED = 42

In [ ]:
np.random.seed(RANDOM_SEED)

## Read Data

In [ ]:
activation_feature_names = [
    'oofp_currie32_cnn_activations',
    'oofp_lystdo_lstm_activations',
]

In [ ]:
magic_feature_lists = [
    'magic_jturkewitz',
    'magic_stas_svd_150',
    'magic_stas_avito',
]

In [ ]:
df_magic_train, df_magic_test, _ = load_feature_lists(magic_feature_lists)

In [ ]:
X_train = np.hstack(
    [
        load(features_data_folder + f'X_train_{nn_name}.pickle')
        for nn_name in activation_feature_names
    ] + [df_magic_train.values]
)

In [ ]:
X_test = np.hstack(
    [
        load(features_data_folder + f'X_test_{nn_name}.pickle')
        for nn_name in activation_feature_names
    ] + [df_magic_test.values]
)

In [ ]:
y_train = load(features_data_folder + 'y_train.pickle')

## Train Models & Compute Out-of-Fold Predictions

In [ ]:
model_params = {
    'num_dense_1': 250,
    'num_dense_2': 120,
    'dense_dropout_rate': 0.5,
}

In [ ]:
def create_dense_block(input_layer, size, dropout_rate, name=None):
    dense = Dense(size)(input_layer)
    bn = BatchNormalization()(dense)
    relu = Activation('relu')(bn)
    dropout = Dropout(dropout_rate, name=name)(relu)
    return dropout

In [ ]:
def create_model(params):
    inputs = Input(shape=(X_train.shape[-1],))
    
    dense_1 = create_dense_block(inputs, params['num_dense_1'], params['dense_dropout_rate'], name='feature_output')
    
    bn_final = BatchNormalization()(dense_2)
    output = Dense(1, activation='sigmoid')(bn_final)
    
    model = Model(
        inputs=inputs,
        outputs=output,
    )

    model.compile(
        loss='binary_crossentropy',
        optimizer='nadam',
        metrics=['accuracy']
    )
    
    return model

In [ ]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [ ]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [ ]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [ ]:
y_test_oofp = np.zeros((len(X_test), NUM_FOLDS))

In [ ]:
for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train, y_train)):   
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model(model_params)
    model.fit(
        X_train[ix_train],
        y_train[ix_train],
        validation_data=(X_train[ix_val], y_train[ix_val]),

        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
        verbose=1,
        
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_train_oofp[ix_val] = model.predict(X_train[ix_val]).reshape(-1)
    y_test_oofp[:, fold_num] = model.predict(X_test).reshape(-1)
    
    # Clear GPU memory.
    K.clear_session()
    del model
    gc.collect()

## Save feature names

In [ ]:
feature_names = [
    'oofp_ubernet',
]

In [ ]:
save_feature_names(feature_names, feature_list_id)

## Save Train features

In [ ]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [ ]:
save_feature_list(y_train_oofp, 'train', feature_list_id)

## Save Test features

In [ ]:
y_test_oofp_mean = np.mean(y_test_oofp, axis=1).reshape((-1, 1))

In [ ]:
save_feature_list(y_test_oofp, 'test', feature_list_id)